In [23]:
%matplotlib inline

In [24]:
import ipywidgets as widgets
import pandas as pd
import html

import base64

import json
import tempfile
import base64
from os import path
import numpy as np




from neurolang.frontend import NeurolangDL, ExplicitVBR, neurosynth_utils
from neurolang import regions

from nilearn import datasets
from nilearn import plotting
import nibabel as nib

from IPython.display import display, Javascript, display_html
from ipysheet import sheet, column, row

In [25]:
nl = NeurolangDL()

@nl.add_symbol
def region_union(rs):
    return regions.region_union(rs)

In [26]:
destrieux_atlas = datasets.fetch_atlas_destrieux_2009()

/Users/dwasserm/anaconda/envs/root3.7/lib/python3.7/site-packages/numpy/lib/npyio.py:2322: VisibleDeprecationWarning: Reading unicode strings without specifying the encoding argument is deprecated. Set the encoding, use None for the system default.
  output = genfromtxt(fname, **kwargs)


In [27]:
destrieux_atlas_image = nib.load(destrieux_atlas['maps'])
destrieux_labels = dict(destrieux_atlas['labels'])

In [28]:
destrieux_set = set()
for k, v in destrieux_labels.items():
    if k == 0:
        continue
    destrieux_set.add((
        v.decode('utf8'),
        ExplicitVBR.from_spatial_image_label(destrieux_atlas_image, k)
    ))

/Users/dwasserm/sources/NeuroLang/neurolang/regions.py:90: DeprecationWarning: get_data() is deprecated in favor of get_fdata(), which has a more predictable return type. To obtain get_data() behavior going forward, use numpy.asanyarray(img.dataobj).

* deprecated from version: 3.0
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 5.0
  data = spatial_image.get_data()


In [29]:
nl.add_tuple_set(destrieux_set, name='destrieux');

In [30]:
class VolumeViewer:
    def __init__(
        self, spatial_images=None, atlas='avg152T1_brain.nii.gz',
        width=800, height=700, frame_name='papayaFrame'
    ):
        self.atlas = atlas
        self.frame_name = frame_name
        self.width = width
        self.height = height
        
        if spatial_images is None:
            spatial_images = []
        self.spatial_images = spatial_images
        
        self.params = {
            'kioskMode': False,
            'worldSpace': True,
            'fullScreen': False
        }
        self.encoder = json.JSONEncoder()
        
    def encode_images(self):
        encoded_images = []
        image_txt = []
        spatial_images = [nib.load(self.atlas)] + self.spatial_images
        for i, image in enumerate(spatial_images):
            encoded_image = base64.encodebytes(nib.Nifti2Image(
                image.get_data(),
                affine=image.affine
            ).to_bytes())
            image_txt.append(f'image{i}')
            enc = encoded_image.decode("utf8").replace("\n", "")
            encoded_images.append(
                f'var {image_txt[-1]}="{enc}";'
            )

        encoded_images = '\n'.join(encoded_images)
        return encoded_images, image_txt

    @property
    def html_representation(self):
        params = dict()
        params.update(self.params)
        
        
        encoded_images, image_names = self.encode_images()
        params['encodedImages'] = image_names
        for image_name in image_names[1:]:
            params[image_name] = {'min': 0, 'max': 10, 'lut': "Red Overlay"}

        if len(self.spatial_images) > 0:
            coords = np.transpose(
                self.spatial_images[-1].get_data().nonzero()
            ).mean(0).astype(int)
            coords = nib.affines.apply_affine(self.spatial_images[-1].affine, coords)
            params['coordinate'] = [int(c) for c in coords]
        
        html = '''
            <!DOCTYPE html>
            <html xmlns="http://www.w3.org/1999/xhtml" lang="en">
                <head>
                    <meta http-equiv="Content-Type" content="text/html; charset=utf-8"/>
                    <meta name="viewport" content="width=device-width, initial-scale=1.0, user-scalable=no"/>
                    <meta name="apple-mobile-web-app-capable" content="yes">
                    <meta name="apple-mobile-web-app-status-bar-style" content="black-translucent">

                    <link rel="stylesheet" type="text/css" href="papaya.css" />
                    <script type="text/javascript" src="papaya.js"></script>
                    <title>Papaya Viewer</title>
                    <script type="text/javascript">

                        {encoded_images}

                        var params={params};
                    </script>
                </head>

                <body>
                    <div style="width:{width}px; height:{height}px;">
                        <div class="papaya" data-params="params"></div>
                    </div>
                </body>
            </html>
        '''
              
        return html.format(
            params=self.encoder.encode(params),
            width=self.width,
            height=self.height,
            encoded_images=encoded_images
        )
    
    @property
    def iframe(self):
        escaped_papaya_html = html.escape(self.html_representation)
        iframe = (
            f'<iframe srcdoc="{escaped_papaya_html}" id="{self.frame_name}" '
            f'width="{self.width}px" height="{self.height}px"></iframe>'
        )
        return iframe

    def reload(self):
        params = dict()
        params.update(self.params)
        encoded_images, image_names = self.encode_images()
        params['encodedImages'] = image_names            
        params = self.encoder.encode(params)
        javascript_script_ = f'''
        var iframe = document.getElementById("{self.frame_name}");
        var papaya_ = iframe.contentWindow.papaya;
        var container = iframe.contentWindow.papaya.Container;

        {encoded_images}

        var params={params};
        console.log(params);
        console.log("start");
        container.resetViewer(0, params);
        console.log("Done");
        '''
        return Javascript(javascript_script_)

In [31]:
class NeurolangWidget:
    def __init__(self, neurolang_engine):
        self.query_widget = widgets.Textarea(
            value='ans(region_union(r)) :- destrieux(..., r)',
            placeholder='Type something',
            #description='String:',
            disabled=False,
            layout = widgets.Layout(display='flex',
                            flex_flow='row',
                            align_items='stretch',
                            width='75%')

        )
        self.button = widgets.Button(description="Run query")
        self.output = widgets.Output()
        
        self.neurolang_engine = neurolang_engine
        self.volume_viewer = VolumeViewer()
        
        self.button.on_click(self.on_query_button_clicked)
        self.box = widgets.VBox([widgets.HTML(self.volume_viewer.iframe)])
        self.number_of_images = 1

        self.widget = widgets.VBox([
            widgets.HBox([
                self.query_widget,
                self.button,
            ]),
            self.box
        ])
        

    def changed(self, event):
        if (
            'name' in event and
            event['name'] == 'value'
        ):
            if  not event['old'] and event['new']:
                self.volume_viewer.spatial_images.append(event['owner'].region_spatial_image)
            elif event['old'] and not event['new']:
                self.volume_viewer.spatial_images.remove(event['owner'].region_spatial_image)
            # self.volume_viewer.reload()
            self.box.children = (
                widgets.HTML(self.volume_viewer.iframe),
            ) + self.box.children[1:]

    def on_query_button_clicked(self, b):
        nl = self.neurolang_engine
        with nl.scope as s:
            nl.execute_nat_datalog_program(self.query_widget.value)
            res = nl.solve_all()
            tt = pd.DataFrame([
                (k, len(v))
                for k, v in res.items()
            ], columns=['set', 'elements'])
        if 'ans' in res:
            ans_items = []
            rows_visible = min(len(res['ans']), 5)
            sheet_ = sheet(
                rows=len(res['ans']), columns=res['ans'].arity,
                layout=widgets.Layout(width='auto', height=f'{50 * rows_visible}px')
            )
            self.volume_viewer.spatial_images = []
            first = True
            for i, tuple_ in enumerate(res['ans'].unwrapped_iter()):
                row_ = []
                for el in tuple_:
                    if isinstance(el, ExplicitVBR):
                        checkbox = widgets.Checkbox(value=first, description='show region')
                        checkbox.papaya_container = self.box
                        checkbox.region_spatial_image = el.spatial_image()
                        checkbox.observe(self.changed)
                        row_.append(checkbox)
                        if first:
                            self.volume_viewer.spatial_images.append(el.spatial_image())
                        first = False
                    else:
                        row_.append(widgets.Label(str(el)))
                    row(i, row_)

            ans_set_widget = widgets.GridBox(
                ans_items,
                layout=widgets.Layout(
                    grid_template_columns=f"repeat({res['ans'].arity}, 100px)"
                )
            )
            
            self.box.children = [
                widgets.HTML(self.volume_viewer.iframe),
                sheet_,
                widgets.HTML(display_html(tt))
            ]
            
nw = NeurolangWidget(neurolang_engine=nl)
nw.widget

/Users/dwasserm/anaconda/envs/root3.7/lib/python3.7/site-packages/ipykernel_launcher.py:28: DeprecationWarning: get_data() is deprecated in favor of get_fdata(), which has a more predictable return type. To obtain get_data() behavior going forward, use numpy.asanyarray(img.dataobj).

* deprecated from version: 3.0
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 5.0


/Users/dwasserm/sources/NeuroLang/neurolang/datalog/aggregation.py:131: UserWarning: No check performed. Should implement check for stratified aggregation
  "No check performed. Should implement check for stratified"
/Users/dwasserm/anaconda/envs/root3.7/lib/python3.7/site-packages/ipykernel_launcher.py:28: DeprecationWarning: get_data() is deprecated in favor of get_fdata(), which has a more predictable return type. To obtain get_data() behavior going forward, use numpy.asanyarray(img.dataobj).

* deprecated from version: 3.0
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 5.0


,set,elements
0,destrieux,150
1,ans,1
